<a href="https://colab.research.google.com/github/george-kan/fastai/blob/main/Mnist_full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#hide
!pip install -Uqq fastbook
import fastbook

In [2]:
#hide
from fastai.vision.all import *
from fastbook import *

In [13]:
path = untar_data(URLs.MNIST)

In [17]:
(path/'training').ls()

(#10) [Path('/root/.fastai/data/mnist_png/training/5'),Path('/root/.fastai/data/mnist_png/training/6'),Path('/root/.fastai/data/mnist_png/training/2'),Path('/root/.fastai/data/mnist_png/training/3'),Path('/root/.fastai/data/mnist_png/training/0'),Path('/root/.fastai/data/mnist_png/training/1'),Path('/root/.fastai/data/mnist_png/training/8'),Path('/root/.fastai/data/mnist_png/training/7'),Path('/root/.fastai/data/mnist_png/training/9'),Path('/root/.fastai/data/mnist_png/training/4')]

In [55]:
train_x = []
train_y = []

In [20]:
folder = 'training/3'

(path/folder).ls()

(#6131) [Path('/root/.fastai/data/mnist_png/training/3/45979.png'),Path('/root/.fastai/data/mnist_png/training/3/39686.png'),Path('/root/.fastai/data/mnist_png/training/3/50007.png'),Path('/root/.fastai/data/mnist_png/training/3/54249.png'),Path('/root/.fastai/data/mnist_png/training/3/21359.png'),Path('/root/.fastai/data/mnist_png/training/3/16793.png'),Path('/root/.fastai/data/mnist_png/training/3/16924.png'),Path('/root/.fastai/data/mnist_png/training/3/1432.png'),Path('/root/.fastai/data/mnist_png/training/3/54339.png'),Path('/root/.fastai/data/mnist_png/training/3/52191.png')...]

In [56]:
for digit in range(10):
    folder = 'training/' + str(digit)
    new_inp = [tensor(Image.open(x)) for x in (path/folder).ls()]
    train_x += new_inp
    train_y += [digit]*len(new_inp)
    

In [57]:
train_x_stacked = torch.stack(train_x).view(-1, 28*28).float()/255
train_y_stacked = tensor(train_y).view(-1, 1)
train_x_stacked.shape, train_y_stacked.shape

AttributeError: ignored

In [60]:
train_x[2000]

tensor([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  71, 101, 237, 254, 255, 254,  92,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 177, 253, 253, 253, 253, 253, 219, 116,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  77, 146, 221, 253, 253, 253, 2

In [62]:
df = pd.DataFrame(torch.trunc(tensor(train_x[2000]).float()))
df.style.set_properties(**{'font-size':'6pt'}).background_gradient('Greys')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,71.000000,101.000000,237.000000,254.000000,255.000000,254.000000,92.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,177.000000,253.000000,253.000000,253.000000,253.000000,253.000000,219.000000,116.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,77.000000,146.000000,221.000000,253.000000,253.000000,253.000000,253.000000,253.000000,253.000000,207.000000,116.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,68.000000,176.000000,253.000000,253.000000,253.000000,253.000000,253.000000,253.000000,253.000000,253.000000,253.000000,142.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,89.000000,237.000000,253.000000,253.000000,253.000000,253.000000,242.000000,214.000000,214.000000,247.000000,253.000000,253.000000,203.000000,76.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,22.000000,139.000000,247.000000,253.000000,253.000000,253.000000,253.000000,253.000000,100.000000,0.000000,0.000000,216.000000,253.000000,253.000000,253.000000,99.000000,0.000000,0.000000,0.000000,0.000000


0